In [1]:
import asyncio
import gzip
import json
import logging
import os
import sys
import urllib
import warnings
from datetime import datetime

import dotenv
import pandas as pd
import requests
from sqlalchemy import create_engine, text

In [7]:
from '/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_model.py' import get_model_version

version = get_model_version()
print("Model version:", version)


SyntaxError: invalid syntax (4168545876.py, line 1)

#### See if IBVStatus Data is ready

In [8]:
import pandas as pd

csv_path = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/ibv_status_data/loonie_shadow_ibvV1.csv"

df_ibv = pd.read_csv(csv_path)

if "IBVStatusID" not in df_ibv.columns:
    raise ValueError("CSV missing 'IBVStatusID' column")

total_rows = len(df_ibv)
unique_ids = df_ibv["IBVStatusID"].nunique(dropna=True)
duplicate_count = total_rows - unique_ids

print(f"Total rows: {total_rows}")
print(f"Unique IBVStatusID: {unique_ids}")
print(f"Duplicate count: {duplicate_count}")

# IDs appearing more than once (including NaN if present)
vc = df_ibv["IBVStatusID"].value_counts(dropna=False)
dupe_ids = vc[vc > 1]
print("\nTop duplicate IDs (count > 1), first 20:")
print(dupe_ids.head(20))

# Show sample rows of duplicates
dupes_df = df_ibv[df_ibv["IBVStatusID"].isin(dupe_ids.index)].sort_values("IBVStatusID")
dupes_df.head(20)


Total rows: 6708
Unique IBVStatusID: 6030
Duplicate count: 678

Top duplicate IDs (count > 1), first 20:
IBVStatusID
62099    44
49850    18
41337    17
27552    15
44865    13
23220    10
15931    10
70076    10
44483     9
26238     9
50227     9
59810     9
76477     9
25831     8
75744     8
7201      8
46530     8
64452     8
15966     7
74845     7
Name: count, dtype: int64


,CModelLogID,ModelName,ApplicationID,PortfolioID,IBVStatusID,DateProcessed,ModelRequest,ModelResponse,ProcessingTime
3981,5290,ibv,0,0,4125,2025-08-21 10:08:18.790,"{""asOfDate"":""2025-08-21"",""accounts"":[{""account...","{""message"":""Unknown error occurred: cannot acc...",2344
3980,5289,ibv,0,0,4125,2025-08-21 10:08:02.797,"{""asOfDate"":""2025-08-21"",""accounts"":[{""account...","{""message"":""Unknown error occurred: cannot acc...",2171
1332,2624,ibv,0,0,4125,2025-06-18 06:54:28.840,"{""asOfDate"":""2025-06-18"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""6gg76em73vIwN3aOD...",3625
677,1794,ibv,0,0,4244,2025-05-23 11:25:17.140,"{""asOfDate"":""2025-05-23"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""wPoOvmamKYUgELV6k...",2577
5073,6382,ibv,0,0,4244,2025-09-23 14:36:27.383,"{""asOfDate"":""2025-09-23"",""accounts"":[{""account...","{""message"":""Unknown error occurred: cannot acc...",1578
1700,3005,ibv,0,0,5078,2025-06-30 11:23:18.000,"{""asOfDate"":""2025-06-30"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""Xgwb1pQLMnu9pjBkz...",3172
1698,3003,ibv,0,0,5078,2025-06-30 11:13:43.120,"{""asOfDate"":""2025-06-30"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""PzzxOngnQ6cVOn76q...",2609
3531,4840,ibv,0,0,5078,2025-08-08 12:07:21.443,"{""asOfDate"":""2025-08-08"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""9YNJvPRZypIoqVLeP...",4111
1736,3041,ibv,0,0,5078,2025-07-01 16:35:47.177,"{""asOfDate"":""2025-07-01"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""XNLRAN6jj4IeDkb3R...",3594
1737,3042,ibv,0,0,5078,2025-07-01 16:39:09.687,"{""asOfDate"":""2025-07-01"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""byM67v7zE6uLMn4ap...",4141


In [9]:
if 'DateProcessed' in df_ibv.columns:
    df_ibv['DateProcessed'] = pd.to_datetime(df_ibv['DateProcessed'])
    df_ibv_sorted = df_ibv.sort_values(["IBVStatusID", "DateProcessed"]).drop_duplicates("IBVStatusID", keep="last")
print(len(df_ibv_sorted)) 

6030


In [10]:
df_ibv_sorted.head()

,CModelLogID,ModelName,ApplicationID,PortfolioID,IBVStatusID,DateProcessed,ModelRequest,ModelResponse,ProcessingTime
380,1315,ibv,0,0,757,2025-05-10 12:42:07.290,"{""asOfDate"":""2025-05-10"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""ErqbnE1VRNhYZzm0r...",3031
4147,5456,ibv,0,0,1258,2025-08-27 08:51:35.213,"{""asOfDate"":""2025-08-27"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""Z1y3oybRZwsyz97P1...",1563
3336,4645,ibv,0,0,1382,2025-08-05 11:45:12.307,"{""asOfDate"":""2025-08-05"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""639pvqA83piYK5gjM...",2546
639,1756,ibv,0,0,1810,2025-05-22 09:35:23.653,"{""asOfDate"":""2025-05-22"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""AeRBY75BDwUm78Mon...",1562
1533,2838,ibv,0,0,2334,2025-06-24 12:28:42.160,"{""asOfDate"":""2025-06-24"",""accounts"":[{""account...","{""accounts"":[{""accountGuid"":""5PeDy5EZ9mtLb6kQD...",2203


#### Save IBV Status to CSV for env file DATAPATH

In [11]:
df_deduped = df_ibv_sorted[['IBVStatusID','DateProcessed']].rename(columns={"DateProcessed": "DateCreated"})
deduped_csv = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/ibv_status_data/loonie_ibv_shadowV1_dedup.csv"
df_deduped.to_csv(deduped_csv, index=False)